In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf


In [2]:
conf = SparkConf()
# conf.set("spark.driver.memory", "4g")
# conf.set("spark.executor.memory", "4g")
conf.setMaster('local[8]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

In [3]:
import json
import math
import itertools

In [4]:
reviews_json = sc.textFile('asnlib/publicdata/train_review.json').map(json.loads)

In [5]:
user_business_pairs = reviews_json.map(lambda x: (x.get('user_id'), x.get('business_id'))).distinct()

In [6]:
user_business_pairs.take(10)

[('OLR4DvqFxCKLOEHqfAxpqQ', 'zK7sltLeRRioqYwgLiWUIA'),
 ('0XMLbsJt-fvcQsOHt3_B_Q', 'eTXYID00jGxq1vZpntBUFw'),
 ('bQCHF5rn5lMI9c5kEwCaNA', 'na4Th5DrNauOv-c43QQFvA'),
 ('fiGqQ7pIGKyZ9G0RqWLMpg', 'IVieU4_z2Y9BsRANZYMNNg'),
 ('sDlRFzGHzfpFHaxvJ1JtTg', 'avkOIpslBUFZMQhELpl3JQ'),
 ('qRU7nYMJIV05lpKQq4jVjw', 'jobP3ywRd3QNZ_GCoPG2DQ'),
 ('d976WkRYJVqJyP3vH2Kayg', 'w6-qlTncaWlSxTJwK_HvfQ'),
 ('v5dxjxKLuINf5E6_kYckXg', 'MgSd4P3ATkiywbbraFygHg'),
 ('mLJ3zQwiEzAIBaXb-7iXBw', '3xykzfVY2PbdjKCRDLdzTQ'),
 ('PQ6-FZpUn2oFLyBGvdiG3w', 'bQ_wtZvMb__OhprY5bF9aQ')]

In [7]:
user_tokens_zip = user_business_pairs.map(lambda x: x[0]).distinct().sortBy(lambda x: x).zipWithIndex()

In [8]:
user_tokens = user_tokens_zip.collectAsMap()

In [9]:
len(user_tokens)

26184

In [10]:
business_tokens_zip = user_business_pairs.map(lambda x: x[1]).distinct().sortBy(lambda x: x).zipWithIndex()

In [11]:
business_tokens = business_tokens_zip.collectAsMap()

In [12]:
len(business_tokens)

10253

In [ ]:
def build_min_hash_func(a,b,p,m):
    def min_hash_func(x):
        return (((a*x + b)%p)%m)
    return min_hash_func

def get_min_hash_functions(num_func, buckets):
    min_hash_func_list = []
    list_a = random.sample(range(1, buckets*100), num_func)
    list_b = random.sample(range(0, buckets*100), num_func)
    p = 6*buckets - 1
    for a,b in zip(list_a, list_b):
        min_hash_func_list.append(build_min_hash_func(a,b,p,buckets))
#         min_hash_func_list.append(lambda x: ((a*x + b)%p)%buckets)
#         print (a,b)
    return min_hash_func_list

In [ ]:
def getMinValue(list1, list2):
    """
    get min value in each element in two list
    :param list1: e.g. [3,6,2,6,8]
    :param list2: e.g. [1,4,5,6,2]
    :return: a list which contain the min value in each column
        e.g.  =======> [1,4,2,6,2]
    """
    return [min(val1, val2) for val1, val2 in zip(list1, list2)]


In [ ]:
min_hash_func_list = get_min_hash_functions(50, len(user_tokens))

In [ ]:
user_hashed_values = user_tokens_zip.map(lambda x: (x[1], [min_hash(x[1]) for min_hash in min_hash_func_list]))

In [ ]:
user_business_tokenized_map = user_business_pairs.map(lambda x: (user_tokens.get(x[0]), business_tokens.get(x[1]))).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [ ]:
user_business_tokenized_map.first()

In [ ]:
business_user_tokenized_map = user_business_pairs.map(lambda x: (business_tokens.get(x[1]), user_tokens.get(x[0]))).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [ ]:
business_user_tokenized_dict = business_user_tokenized_map.collectAsMap()

In [ ]:
len(business_user_tokenized_dict[4396])

In [ ]:
signature_matrix_rdd = user_business_tokenized_map.leftOuterJoin(user_hashed_values)

In [ ]:
signature_matrix_rdd = signature_matrix_rdd.map(lambda kvv: kvv[1])

In [ ]:
signature_matrix_rdd = signature_matrix_rdd.flatMap(lambda bidx_uhidxs: [(bidx, bidx_uhidxs[1]) for bidx in bidx_uhidxs[0]]) \

In [ ]:
signature_matrix_rdd = signature_matrix_rdd.reduceByKey(getMinValue).coalesce(2)

In [ ]:
signature_matrix_rdd.take(10)

In [ ]:
signature_matrix_dict = signature_matrix_rdd.collectAsMap()

In [ ]:
[list(range(0,50))[i:i + 5] for i in range(0, 50, 5)]